## _Medical Insurance Costs_

Pada kasus ini, terdapat data tentang informasi kesehatan dan biaya yang harus dikeluarkan oleh asuransi kesehatan. Infomasi terkait dengan data _medical insurance cost_ adalah sebagai berikut,

1. Age: Usia penerima manfaat
2. Sex: Gender penerima manfaat (_male_, _femele_)
3. Bmi : Body Mass Index
4. Children: Jumlah anak/tanggungan yang dicover oleh pihak asuransi
5. Smoker: Status perokok (_yes_, _no_)
6. Region: Wilayah tempat tinggal penerima manfaat
7. Charges: Biaya yang dikeluarkan oleh asuransi

In [97]:
# Selayang pandang data Medical Insurance Costs
import pandas as pd

df = pd.read_csv('data/insurance.csv')

display(df.head())

display(df.corr())

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


,age,bmi,children,charges
age,1.000000,0.109272,0.042469,0.299008
bmi,0.109272,1.000000,0.012759,0.198341
children,0.042469,0.012759,1.000000,0.067998
charges,0.299008,0.198341,0.067998,1.000000


### Tantangan

Buatlah model regresi untuk memprediksi biaya yang harus dikeluarkan oleh pihak asuransi berdasarkan data. Validasi performa model regresi Anda dengan nilai ***R-squared ($R^2$)***

#### _Tasks_

1. Pastikan semua variabel kategorial diolah dengan baik. (Gunakan fitur mapping pada pandas)
2. Cek kondisi multicollinearity untuk semua variabel independen. Jika ada, antar variabel apakah itu?
3. Pastikan model menggunakan variabel yang tidak memiliki nilai multicollinearity yang tinggi
4. (Hints) Anda dapat menggunakan nilai ***Variance Inflation Factor (VIF)*** untuk mengetahui tingkat multicollinearity pada sebuah variabel independent.
5. Evaluasi model yang Anda buat dengan nilai $R^2$
6. Simpulkan, variabel independen apa saja yang dapat digunakan untuk menghasilkan model regresi yang baik pada kasus _medical insurance costs_?

#### (Hints) Interpretasi Nilai VIF

- 1 - variabel indenpenden tidak memiliki korelasi dengan variabel independen yang lain
- 1 < VIF < 5 - variabel independen sedikit memiliki korelasi dengan variabel independen yang lain
- VIF > 5 - variabel independen memiliki korelasi yang kuat dengan variabel independen lainnya
- VIF > 10 - variabel independen miliki korelasi yang sangat kuat dengan variabel independen dan perlu diperhatikan lebih lanjut

#### (Hints) Implementasi Perhitungan VIF

VIF dapat dihitung secara langsung dengan menggunakan library dari `statsmodels`

#### (Hints) Scatterplot Korelasi Antar Variabel

![var_cor](assets/var_corr.png)

## JAWABAN

In [98]:
# Mendeklarasikan data baru
df2 = pd.read_csv('data/insurance.csv')

display(df2.head())

display(df2.corr())

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


,age,bmi,children,charges
age,1.000000,0.109272,0.042469,0.299008
bmi,0.109272,1.000000,0.012759,0.198341
children,0.042469,0.012759,1.000000,0.067998
charges,0.299008,0.198341,0.067998,1.000000


#### Mengubah data tipe kategorik menjadi data dalam bentuk numerik

In [99]:
# Kolom sex
labels_sex = {
    'female': 0, 
    'male': 1
}

df2['sex'] = df2['sex'].map(labels_sex)
df2.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,yes,southwest,16884.92400
1,18,1,33.770,1,no,southeast,1725.55230
2,28,1,33.000,3,no,southeast,4449.46200
3,33,1,22.705,0,no,northwest,21984.47061
4,32,1,28.880,0,no,northwest,3866.85520


In [100]:
# Kolom region
labels_region = {
    'southwest': 0, 
    'southeast': 1,
    'northwest': 2,
    'northeast': 3
}

df2['region'] = df2['region'].map(labels_region)
df2.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,yes,0,16884.92400
1,18,1,33.770,1,no,1,1725.55230
2,28,1,33.000,3,no,1,4449.46200
3,33,1,22.705,0,no,2,21984.47061
4,32,1,28.880,0,no,2,3866.85520


In [101]:
# Kolom smoker
labels_smoker = {
    'no': 0, 
    'yes': 1
}

df2['smoker'] = df2['smoker'].map(labels_smoker)
df2.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,0,16884.92400
1,18,1,33.770,1,0,1,1725.55230
2,28,1,33.000,3,0,1,4449.46200
3,33,1,22.705,0,0,2,21984.47061
4,32,1,28.880,0,0,2,3866.85520


#### Menggunakan nilai Variance Inflation Factor (VIF) untuk mengetahui tingkat multicollinearity

Dari hasil pengecekan, data yang memiliki tingkat multicolinearity yang tinggi ada pada children, smoker, sex, dan region

In [102]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = df2[['age', 'sex', 'bmi', 'children', 'smoker', 'region']]

vif_check = pd.DataFrame()
vif_check['feature'] = X.columns

vif_check['VIF'] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
print(vif_check)

    feature       VIF
0       age  7.752935
1       sex  2.006067
2       bmi  9.243496
3  children  1.801371
4    smoker  1.258097
5    region  2.527799


##### Menghapus kolom yang memiliki tingkat multicolinearity tinggi

In [103]:
df_copy = df2.copy()
df_copy = df_copy.drop(df_copy.columns[0], axis=1)
df_copy = df_copy.drop(df_copy.columns[2], axis=1)
df_copy.head()

,sex,bmi,smoker,region,charges
0,0,27.900,1,0,16884.92400
1,1,33.770,0,1,1725.55230
2,1,33.000,0,1,4449.46200
3,1,22.705,0,2,21984.47061
4,1,28.880,0,2,3866.85520


In [104]:
df3 = df_copy.copy()

In [105]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [106]:
X = df3.iloc[:, :-1].values
y = df3.iloc[:, 4].values

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.2, random_state=50)

##### Evaluasi model dengan nilai R2

In [107]:
from sklearn.linear_model import LinearRegression

liner = LinearRegression()
liner.fit(X_train, y_train)
y_pred = liner.predict(X_test)

In [115]:
conc = np.concatenate((y_test, y_pred))
conc

array([ 5976.8311    ,  5846.9176    , 13831.1152    , ...,
       30076.82103861,  7382.84012218,  7260.39026268])

In [109]:
from sklearn.metrics import r2_score

rsquare = r2_score(y_test, y_pred)
rsquare

0.6502361973910064

##### Menghitung nilai R2 pada semua kolom

In [110]:
X2 = df2.iloc[:, :-1].values
y2 = df2.iloc[:, 6].values

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, train_size=0.2, random_state=50)

In [111]:
y2 = y2.reshape(len(y), 1)
y2.shape

(1338, 1)

In [94]:
liner2 = LinearRegression()
liner2.fit(X_train2, y_train2)

y_pred2 = liner2.predict(X_test2)

In [112]:
conc2 = np.concatenate((y_test2, y_pred2))
conc2

array([ 5976.8311    ,  5846.9176    , 13831.1152    , ...,
       31009.67015436,  2087.9179576 ,  5809.81519562])

In [113]:
rsquare2 = r2_score(y_test2, y_pred2)
print(rsquare2)

0.7404617861795864


#### Kesimpulan
R square disebut juga sebagai koefisien determinasi yang menjelaskan seberapa jauh data dependen dapat dijelaskan oleh data independen. R square bernilai antar 0 – 1 dengan ketentuan semakin mendekati angka satu berarti semakin baik. 
Dari hasil diatas, dapat dilihat bahwa pengecekan pada semua kolom mendapatkan nilai yang lebih baik daripada pengecekan pada beberapa kolom tertentu saja.